In [1]:
"""Plots for Hydrogen Mass function for all Snaps"""

# import the packages we need
import numpy as np  # work horse package for numerical work in python
import matplotlib.pyplot as plt # plotting library
from dragons import meraxes, munge# DRAGONS modules for reading and dealing with model ouput
from Functions import *

fname_in = "imapping_testrun/meraxes.hdf5" # Get the file name for the meraxes file we are using
h = meraxes.set_little_h(0.7)# By specifying Hubble parameter: h=0.7, we are ensuring that all of the galaxy properties

# Read in all the snap shots using the function 'read snaplist'. Little h previously set as 0.7
snaplist = meraxes.io.read_snaplist(fname_in, h)
snaps = snaplist[0]
redshifts=snaplist[1]
lt_times=snaplist[2]
# Print the values if you want to see
print ('snaps: ', snaps)
print('redshifts: ', redshifts)
print('light travel time: ', lt_times)


#plt.title("Hydrogen Mass Function for all Snap Shots")
'''     
# You can also see the units of all galaxy properties (and grids properties if
# they are present).  Here I am only printing the galaxy properties.

units = meraxes.read_units(fname_in)
print('\nunits\n=====')
for k, v in units.items():
    if not isinstance(v, dict):
        print(k, ':', v.decode('utf-8'))
print()
'''

snaps = list(snaps)
# for each snapshot...


snaps:  [100 115 134 158 173 192 216 250]
redshifts:  [4.9999988  4.01575197 2.99733298 2.00100053 1.50871276 0.99933302
 0.50100094 0.        ]
light travel time:  [12231.24679633 11881.54868823 11283.32371736 10185.99335738
  9241.64988071  7682.30347798  5035.97124274     0.        ]


In [2]:
# GET DATA FROM FILES
# ZWAAN 2005 DATA SET
file_Zwaan = open("data/HIMF_Zwaan2005.data", "r")
fileread_Zwaan = file_Zwaan.readlines()
for line in fileread_Zwaan:
    if line.startswith("#"):
        continue # skips that iteration 

data_Zwaan = np.loadtxt(fileread_Zwaan, delimiter=" ")
#print(data_Zwaan)
log_MHI_Zwaan = data_Zwaan[:,0]
print('Zwaan 2005')
print('log_MHI : ', log_MHI_Zwaan)
log_HIMF_Zwaan = data_Zwaan[:,1]
print('log_HIMF : ', log_HIMF_Zwaan)

# MARTIN 2010 DATA SET FOR 1/ V_max Method
file_Martin_VMAX = open("data/HIMF_Martin2010_1-VMAX.data", "r")# from here on Data from Martin 2010 1/VMax method will be referred to as data_MV
fileread_MV = file_Martin_VMAX.readlines()
for line in fileread_MV:
    if line.startswith("#"):
        continue # skips that iteration 

data_MV = np.loadtxt(fileread_MV, delimiter="  ") 
#print(data_Zwaan)
log_MHI_MV = data_MV[:,0]
print('Martin 1/ V_{max} 2010')
print('log_MHI : ', log_MHI_MV)
log_HIMF_MV = data_MV[:,1]
print('log_HIMF : ', log_HIMF_MV)




FileNotFoundError: [Errno 2] No such file or directory: 'data/HIMF_Zwaan2005.data'

In [ ]:
# CREATE EMPTY FIGURE
fig, ax = plt.subplots(1, 1)

for i in snaps:
    
    # Get galaxy catalogue
    gals, sim_props = meraxes.read_gals(fname_in, i, sim_props=True) #read in galaxies
    # The units of HI Mass ouput by the model are 1e10 Msol.  Let's convert
    # this to log10(M/Msol).
    gals["HIMass"] = np.log10(gals["HIMass"]*1e10)
    #print('gals HI Mass: ', gals["HIMass"])

    # HImf = "HIMass" munge function used
    HImf = ALICE_mass_function(gals["HIMass"], sim_props["Volume"], bins=50,
                             range=(5.5, 11.5))

    # Plot the HI mass function
    snapshot_used = str(i)
    #print(snapshot_used)
    snaplist_index_used = snaps.index(i)# convert to list and find the index of the snap we are using
    redshift_used = snaplist[1][snaplist_index_used] # Get the redshift for corresponding snapshot
    redshift_used = round(redshift_used, 1) # round float to nearest whole int
    #print(redshift_used)
    redshift_used=str(redshift_used)
    plot_number = ('plot_' + redshift_used)

    # Change color depending on redshift
    if redshift_used == '0.0':
        color_plot = 'darkred'
    elif redshift_used == '0.5':
        color_plot = 'orangered'
    elif redshift_used == '1.0':
        color_plot = 'orange'
    elif redshift_used == '1.5':
        color_plot = 'gold'
    elif redshift_used == '2.0':
        color_plot = 'limegreen'
    elif redshift_used == '3.0':
        color_plot = 'cyan'
    elif redshift_used == '4.0':
        color_plot = 'royalblue'
    else:
        color_plot = 'darkblue'

    #print(plot_number)
    ax.plot(HImf[:, 0], np.log10(HImf[:, 1]), color=color_plot, lw=2,
            label=("z = " + redshift_used))

# PLOT THE DATA
label_MARTIN = ("MARTIN 2010 (1/$V_{max}$)")
label_ZWAAN =('ZWAAN 2005')


# MARTIN 2010
ax.plot(data_MV[:,0], data_MV[:,1], marker='o',color='dimgray',lw=1,
        label=label_MARTIN)
ax.errorbar(data_MV[:,0], data_MV[:,1],xerr=None,yerr=data_MV[:,2], 
            color='dimgray',elinewidth=1,capsize=3)

# ZWAAN 2005
ax.plot(data_Zwaan[:,0], data_Zwaan[:,1], marker='s',color='lightgray',
        lw=1, label=label_ZWAAN) # change to np.random.rand(3,) for random colours
ax.errorbar(data_Zwaan[:,0], data_Zwaan[:,1], xerr=data_Zwaan[:,2], yerr=data_Zwaan[:,3],
            color='lightgray',elinewidth=1,capsize=3)

# Add the legend
ax.legend(title=("DRAGONS"), loc="lower left")

# Set the axis labels.
# Note the use of LaTeX here.
ax.set_xlabel(r"$\log_{10}(M_{HI} / {\rm M_{\odot}})$")
ax.set_ylabel(r"$\log_{10}(\phi / {\rm Mpc^{-3}})$")

# Finally save the figure as a PDF
plt.tight_layout()

name_of_plot=("HIMF_allz_wDATA")
plt.savefig("plots/wDATA/" + name_of_plot + ".pdf")

In [ ]:
# CREATE EMPTY FIGURE
fig, ax = plt.subplots(1, 1)

# HImf = "HIMass" munge function used
HImf = ALICE_mass_function(gals["HIMass"], sim_props["Volume"], bins=50,
                         range=(5.5, 11.5))

# Plot the HI mass function
snapshot_used = str(i)
#print(snapshot_used)
snaplist_index_used = snaps.index(i)# convert to list and find the index of the snap we are using
redshift_used = snaplist[1][snaplist_index_used] # Get the redshift for corresponding snapshot
redshift_used = round(redshift_used, 1) # round float to nearest whole int
#print(redshift_used)
redshift_used=str(redshift_used)
plot_number = ('plot_' + redshift_used)


#print(plot_number)
ax.plot(HImf[:, 0], np.log10(HImf[:, 1]), color='darkred', lw=2,
        label=("z = " + redshift_used))

# Add the legend
ax.legend(title=("DRAGONS"), loc="lower left")

# Set the axis labels.
# Note the use of LaTeX here.
ax.set_xlabel(r"$\log_{10}(M_{HI} / {\rm M_{\odot}})$")
ax.set_ylabel(r"$\log_{10}(\phi / {\rm Mpc^{-3}})$")

# Finally save the figure as a PDF
plt.tight_layout()

name_of_plot=("HIMF_allz_wDATA")
plt.savefig("plots/wDATA/" + name_of_plot + ".pdf")